In [160]:
import re 
import pandas as pd 
import os.path

# READING MOVES FROM FILE TO DARALOGIC 

def Update_CSV(data,final_file_name):
    moves = re.findall('(.*)\n',data)

    last = '' # last line read 

    tournament_name = city = ''
    year = month = date = game_no = 0
    player_white = player_black = ''
    elo_white = elo_black = 0
    result = -2
    FEN = turn_color = ''
    best_eval = 0
    played_eval = 0
    eval_diff = 0


    all_rows = []

    for i in moves:
        if(last==''):
            # new move starts 
            #print(i)
            row = i.split(';')
            #print(row)
            tournament_name = re.findall('#\d+ (\D*)\s*.*',row[0])[0]
            tournament_name = tournament_name.strip()

            city = row[1]
            year,month,date = [i for i in row[2].split('.')[0:3]]
                            
            dummy = row[3] # dont know for sure 
            player_white = row[4].replace(',',' ')
            player_black = row[5].replace(',',' ')

            if('1/2' in row[6]):
                result = 0.5
            elif ('1-0' in row[6]):
                result = 1
            else:
                result = 0

            #print(tournament_name,city,year,month,date,dummy,player_white,player_black,result)

            last = i


        else:


            #print(i)
            last = i 
            if(last==''):
                continue

            FEN = i.split(' ')[0]
            QW = len(re.findall('Q',FEN))
            QB = len(re.findall('q',FEN))
            RW = len(re.findall('R',FEN))
            RB = len(re.findall('r',FEN))
            BW = len(re.findall('B',FEN))
            BB = len(re.findall('b',FEN))
            KW = len(re.findall('N',FEN))
            KB = len(re.findall('n',FEN))
            #print(QW,QB,RW,RB,BW,BB,KW,KR)


            turn_color = i.split(' ')[1]
            if(turn_color == 'w'):
                turn_color = 1
            else:
                turn_color = -1

            dummy1 = i.split(' ')[4]
            row = i.split(' ')[5].split(',')
            #print(row)
            move_number = int(row[0])
            if('1/2' in row[2]):
                result = 0
            elif ('1-0' in row[2]):
                result = 1
            else:
                result = -1

            #print(result)
            if(turn_color==1):
                elo_white = row[3]
                elo_black = row[4]
            else:
                elo_black = row[3]
                elo_white = row[4]


            best_eval = re.findall('\d+',row[7])[0]
            sign_best_eval = 1
            if('-' in row[7]):
                sign_best_eval = -1

            played_eval = re.findall('\d+',row[9])[0]
            sign_played_eval = 1
            if('-' in row[9]):
                sign_played_eval = -1



            if(best_eval=='000'):
                best_eval = 0
            else:
                best_eval = int(best_eval.lstrip('0'))
            if(played_eval=='000'):
                played_eval = 0
            else:
                played_eval = int(played_eval.lstrip('0'))

            best_eval = best_eval*sign_best_eval #sign makes it whatever sign it is and then turn color makes it positive for black
            played_eval = played_eval*sign_played_eval
            eval_diff = abs(best_eval-played_eval)
            #print(row[7],best_eval,row[9],played_eval,i.split(' ')[1])

            #if(bes)
            df_row = [tournament_name, city, year, player_white, player_black,elo_white, elo_black,\
                    turn_color, move_number, QW, QB, RW, RB, BW, BB, KW, KB, 
                    best_eval, played_eval, eval_diff, result]
            #print(df_row)
            all_rows.append(df_row)

            colnames = ['Tournament','City','Year','Player_White','Player_Black','ELO_White','ELO_Black',\
                        'Turn','Move#','QueenWhite', 'QueenBlack', 'RookWhite', 'RookBlack','BishopWhite',\
                        'BishopBlack','KnightWhite','KnightBlack','Best_Eval','Played_Eval','Eval_Diff','Result']

    
    new_data = pd.DataFrame(all_rows,columns=colnames)
    if(os.path.isfile(final_file_name)):
        original_data = pd.read_csv(final_file_name)
        updated_data = original_data.append(new_data,ignore_index=True)
        updated_data = updated_data.drop_duplicates(keep=False)
        updated_data.to_csv(final_file_name,index = False)
    else:
        new_data.to_csv(final_file_name,index = False)

In [ ]:
# Read files 
files = ['AA1025SF7moves.txt','AA1050SF7moves.txt','AA1075SF7moves.txt','AA1100SF7moves.txt','AA1125SF7moves.txt','AA1150SF7moves.txt','AA1175SF7moves.txt','AA1200SF7moves.txt','AA1225SF7moves.txt','AA1250SF7moves.txt','AA1275SF7moves.txt','AA1300SF7moves.txt','AA1325SF7moves.txt','AA1350SF7moves.txt','AA1375SF7moves.txt','AA1400SF7moves.txt','AA1425SF7moves.txt','AA1450SF7moves.txt','AA1475SF7moves.txt','AA1500SF7moves.txt','AA1525SF7moves.txt','AA1550SF7moves.txt','AA1575SF7moves.txt','AA1600SF7moves.txt','AA1625SF7moves.txt','AA1650SF7moves.txt','AA1675SF7moves.txt','AA1700SF7moves.txt','AA1725SF7moves.txt','AA1750SF7moves.txt','AA1775SF7moves.txt','AA1800SF7moves.txt','AA1825SF7moves.txt','AA1850SF7moves.txt','AA1875SF7moves.txt','AA1900SF7moves.txt','AA1925SF7moves.txt','AA1950SF7moves.txt','AA1975SF7moves.txt','AA2000SF7moves.txt','AA2025SF7moves.txt','AA2050SF7moves.txt','AA2075SF7moves.txt','AA2100SF7moves.txt','AA2125SF7moves.txt','AA2150SF7moves.txt','AA2175SF7moves.txt','AA2200SF7moves.txt','AA2225SF7moves.txt','AA2250SF7moves.txt','AA2275SF7moves.txt','AA2300SF7moves.txt','AA2325SF7moves.txt','AA2350SF7moves.txt','AA2375SF7moves.txt','AA2400SF7moves.txt','AA2425SF7moves.txt','AA2450SF7moves.txt','AA2475SF7moves.txt','AA2500SF7moves.txt','AA2525SF7moves.txt','AA2550SF7moves.txt','AA2575SF7moves.txt','AA2600SF7moves.txt','AA2625SF7moves.txt','AA2650SF7moves.txt','AA2675SF7moves.txt','AA2700SF7moves.txt','AA2725SF7moves.txt','AA2750SF7moves.txt','AA2775SF7moves.txt','AA2800SF7moves.txt','AA2825SF7moves.txt','RBZ14SF7moves.txt']
print(len(files))
cnt = 1
for i in files:
    with open('MOVES/' + i, 'r') as myfile:
        print('\n\n\n',cnt,i,'\n\n\n')
        cnt += 1
        data = myfile.read()
        Update_CSV(data,'Chess_corrected_2.csv')

In [ ]:
from sklearn.utils import shuffle

# SHUFFLE the file --> run this only once so your data remains the same
def shuffler(final_file_name):
    net_data = pd.read_csv(final_file_name)
    len(net_data)
    net_data = shuffle(net_data)
    net_data.to_csv(final_file_name,index = False)

    
#shuffler('Chess_corrected.csv')
    

In [ ]:
main_data = pd.read_csv('Chess_corrected.csv')

In [152]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# SELECTING THE SAMPLE OF ~ 1 MILLION BECAUSE 7 MILLION DOESNT SEEM POSSIBLE RIGHT NOW 

def filtering(sample):
    sample = sample.drop(['Player_White','Player_Black','Tournament','City'],axis = 1) # this is temporary because 

    # count number of row with unr in elo 
    initial = len(sample)
    sample = sample[sample['ELO_Black'].apply(lambda x: str(x).isdigit())]
    sample = sample[sample['ELO_White'].apply(lambda x: str(x).isdigit())]
    resulting = len(sample)
    
    #print(initial-resulting , ' rows dropped')
    return sample
'''
#sample = main_data.loc[1:1000000]
sample = pd.read_csv('sample.csv')

# we did not get time to process data as of now 


sample = filtering(sample)

sample_train, sample_test = train_test_split(sample, test_size=0.2)
sample_test, sample_validation = train_test_split(sample_test, test_size=0.5)



x_sample_train = sample_train.drop('Result',axis = 1)
y_sample_train = sample_train['Result']

x_sample_validation = sample_validation.drop('Result',axis = 1)
y_sample_validation = sample_validation['Result']

x_sample_test = sample_test.drop('Result',axis = 1)
y_sample_test = sample_test['Result']

'''
    
    

"\n#sample = main_data.loc[1:1000000]\nsample = pd.read_csv('sample.csv')\n\n# we did not get time to process data as of now \n\n\nsample = filtering(sample)\n\nsample_train, sample_test = train_test_split(sample, test_size=0.2)\nsample_test, sample_validation = train_test_split(sample_test, test_size=0.5)\n\n\n\nx_sample_train = sample_train.drop('Result',axis = 1)\ny_sample_train = sample_train['Result']\n\nx_sample_validation = sample_validation.drop('Result',axis = 1)\ny_sample_validation = sample_validation['Result']\n\nx_sample_test = sample_test.drop('Result',axis = 1)\ny_sample_test = sample_test['Result']\n\n"

In [64]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import pickle

# MACHINE LEARNING MODELS 

def random_forest_cl(x_sample_train,y_sample_train,x_sample_test,y_sample_test):
    rfc = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=123456)
    rfc.fit(x_sample_train, y_sample_train)
    filename_rfc = 'rfcModel.sav'
    pickle.dump(rfc, open(filename_rfc, 'wb'))
    predicted = rfc.predict(x_sample_test)
    accuracy = accuracy_score(y_sample_test, predicted)
    cm = metrics.confusion_matrix(y_sample_test, predicted)
    print(accuracy,'\n',cm)
    

def logisitic_cl(x_sample_train,y_sample_train,x_sample_test,y_sample_test):
    lrc = LogisticRegression()
    lrc.fit(x_sample_train, y_sample_train)
    predicted = lrc.predict(x_sample_test)
    accuracy = accuracy_score(y_sample_test, predicted)
    cm = metrics.confusion_matrix(y_sample_test, predicted)
    print(accuracy,'\n',cm)


In [110]:
#random_forest_cl(x_sample_train,y_sample_train,x_sample_test,y_sample_test)

0.6566065660656607 
 [[16373  7243  3740]
 [ 4554 25989  6991]
 [ 2827  8984 23298]]


In [2]:
import pickle
loaded_model = pickle.load(open('rfcModel.sav', 'rb'))

In [6]:
predicted = loaded_model.predict(x_sample_validation)

In [12]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
accuracy = accuracy_score(y_sample_validation, predicted)
cm = metrics.confusionmatrix(y_sample_validation, predicted)
print(accuracy,'\n',cm)


0.93167 
 [[25379  1440   705]
 [  904 34806  1371]
 [  586  1827 32982]]


In [62]:


temp = temp[temp['Move#'] > 0]
print(len(temp))
temp = temp.drop('Unnamed: 0', axis = 1)
temp = temp.drop(['Player_White','Player_Black','Tournament','City'],axis = 1)

tx = temp.drop('Result',axis = 1)
ty = temp['Result']
predicted = loaded_model.predict(tx)


7695


In [158]:

elow = [1200,1800,2400,2700] 
ehigh = [1800,2400,2700,3100]


for i in range(0,len(elow)):
    VM = main_data.loc[2000000:2030000]
    VM = filtering(VM)
    #VM = VM[(VM['ELO_White'] < ehigh[i]) & (VM['ELO_White'] > elow[i]) & (VM['Move#']>=25) & (VM['Move#']<35)]
    print(len(VM),end = ',')
    X_VM = VM.drop(['Result','Unnamed: 0'],axis = 1)
    Y_VM = VM['Result']
    predicted = loaded_model.predict(X_VM)
    accuracy = accuracy_score(Y_VM, predicted)
    cm = metrics.confusion_matrix(Y_VM, predicted)
    print(accuracy, end = ' ')



719,0.7329624478442281 3574,0.6667599328483492 2147,0.7177456916627852 195,0.7538461538461538 

In [87]:
main_data.loc[200000:200100]

,Unnamed: 0,Tournament,City,Year,Player_White,Player_Black,ELO_White,ELO_Black,Turn,Move#,...,RookWhite,RookBlack,BishopWhite,BishopBlack,KnightWhite,KnightBlack,Best_Eval,Played_Eval,Eval_Diff,Result
200000,200000,OHRA-B,Amsterdam,1986,Hartoch Robert G,Piket Marcel,2370,2375,1,57,...,0,0,0,0,1,1,192,192,0,1
200001,200001,RUS-ch (Women),Orel,2006,Tairova Elena,Gunina Valentina,2342,2352,-1,41,...,1,0,1,0,0,2,577,577,0,-1
200002,200002,Brno II etapa-B,Brno,2004,Sevcik Jakub,Licka Vlastimil,1895,1909,1,10,...,2,2,2,2,2,2,-34,-34,0,0
200003,200003,Wch U,Halle,1995,Spangenberg Hugo,Fridman Daniel,2505,2490,1,21,...,2,2,0,2,1,0,-22,-22,0,1
200004,200004,Poikovsky Karpov,Poikovsky,2017,Najer Evgeniy,Anton Guijarro David,2682,2679,1,15,...,2,2,2,2,2,2,-14,-14,0,0
200005,200005,CZE-chTV,Czechia,2001,Kolar Pavel,Ruzicka Zdenek,2125,2125,1,24,...,2,2,0,0,0,0,-80,-80,0,-1
200006,200006,Florianopolis op,Florianopolis,2017,Lourenco dos Santos Paulo Rogerio,Do Prado Hamilton Mesquita,1719,1732,1,34,...,2,2,0,1,1,1,-20,-31,11,0
200007,200007,EU-Cup,Kemer,2007,Zifroni Dov,Kalod Radek,2490,2490,-1,44,...,0,0,0,0,1,1,-181,-289,108,1
200008,200008,Caribbean op,Port of Spain,2009,Corbin Philip,Suez Panama Gilles,2125,2122,-1,32,...,1,2,2,1,0,0,-1159,-1160,1,1
200009,200009,Tagaytay City zt,Tagaytay City,2017,Liu Xiangyi,Yeoh Li Tian,2443,2449,-1,35,...,1,1,0,0,0,0,1550,1550,0,-1
